In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
# data load
seoul = gpd.read_file('data/seoul_geo/seoul.shp').to_crs(epsg=5174)
roads = gpd.read_file('data/seoul_road/Z_KAIS_TL_SPRD_MANAGE_11.shp', encoding='euc-kr', crs='epsg:5179')
roads = roads.to_crs('epsg:5174')
roads = roads[roads.ROA_CLS_SE.isin(['1','2','3'])]
grid = pd.read_pickle('data/grid.pkl')

# Select grid cells that intersect with the roads

roads_union = roads.unary_union
grid_roads = grid[grid.intersects(roads_union)]

In [68]:
grid_roads.to_pickle('data/grid_roads.pkl')

In [3]:
flood = gpd.read_file('data/flood/서울시_2022.shp').to_crs(epsg=5174)

flood.F_SAT_YMD = flood.F_SAT_YMD.str.replace('-','')
flood.F_SAT_YMD = flood.F_SAT_YMD.astype(int)
flood.F_END_YMD = flood.F_END_YMD.str.replace('-','')
flood.F_END_YMD = flood.F_END_YMD.astype(int)

# to datetime
flood['START'] = pd.to_datetime(flood['F_SAT_YMD'], format='%Y%m%d') + pd.to_timedelta(flood.F_SAT_TM.astype(int), unit='h')
flood['END'] = pd.to_datetime(flood['F_END_YMD'], format='%Y%m%d') + pd.to_timedelta(flood.F_END_TM.astype(int), unit='h')

flood = flood[['START', 'END', 'geometry']]

start_date = pd.to_datetime('2022-06-01')
end_date = pd.to_datetime('2023-01-01')
time_index = pd.date_range(start_date, end_date, freq='h')

hourly_df = pd.DataFrame(index=time_index)
hourly_df['geometry'] = None

for _, row in tqdm(flood.iterrows(), total=flood.shape[0]):
    start_time = row['START']
    end_time = row['END']
    geometry = row['geometry']

    # Create a mask for the time index within the start and end times
    mask = (hourly_df.index >= start_time) & (hourly_df.index <= end_time)

    # Update the 'geometry' column using the mask
    hourly_df.loc[mask, 'geometry'] = hourly_df.loc[mask, 'geometry'].apply(lambda x: x.union(geometry) if x is not None else geometry)

  0%|          | 0/8668 [00:00<?, ?it/s]

In [ ]:
hourly_df.to_csv('data/hourly_flood.csv')

,START,END,geometry
0,2022-08-08 18:00:00,2022-08-08 23:00:00,"POLYGON ((200449.162 463357.569, 200448.918 46..."
1,2022-08-08 18:00:00,2022-08-08 23:00:00,"POLYGON ((200615.226 463266.483, 200599.392 46..."
2,2022-08-08 18:00:00,2022-08-08 23:00:00,"POLYGON ((200409.422 462958.018, 200500.022 46..."
3,2022-08-08 20:00:00,2022-08-11 00:00:00,"POLYGON ((191953.847 446216.637, 191954.856 44..."
4,2022-08-08 18:00:00,2022-08-08 23:00:00,"POLYGON ((202858.165 457933.427, 202863.633 45..."
...,...,...,...
8663,2022-08-08 21:00:00,2022-08-09 05:00:00,"POLYGON ((197093.635 450267.283, 197100.651 45..."
8664,2022-08-08 21:00:00,2022-08-09 05:00:00,"POLYGON ((200428.979 447976.959, 200427.063 44..."
8665,2022-08-08 21:00:00,2022-08-09 05:00:00,"POLYGON ((195925.244 448185.792, 195927.687 44..."
8666,2022-08-08 19:00:00,2022-08-10 12:00:00,"POLYGON ((201910.741 454875.075, 201912.201 45..."


In [57]:
# For each grid cell and each time period, check if the flood event intersects with the grid cell

start_date = flood['START'].min()
end_date = flood['START'].max()

date_range = pd.date_range(start_date, end_date, freq='h')

result = 

joined = gpd.sjoin(grid_roads, flood, how='left', op='intersects')
joined = joined

/Users/dangchan/miniforge3/envs/colab/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Index(['geometry', 'ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE', 'flood',
       'office_cat_0', 'office_cat_1', 'office_cat_2', 'office_cat_3',
       'office_cat_4', 'office_cat_5', 'office_cat_6', 'office_cat_7',
       'office_cat_8', 'office_cat_9', 'elevation', 'rainpump', 'trees',
       'index_right', 'START', 'END'],
      dtype='object')